In [1]:
import model as m
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
import os

os.environ['TF_GPU_ALLOCATOR']='cuda_malloc_async'
os.environ["TF_CPP_VMODULE"]="gpu_process_state=10,gpu_cudamallocasync_allocator=10"

import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.set_logical_device_configuration(
                gpu,
                [tf.config.LogicalDeviceConfiguration(memory_limit=256)])

        logical_gpus = tf.config.experimental.list_logical_devices('GPU')

    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

2022-03-23 17:01:56.085748: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-23 17:01:56.093245: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-23 17:01:56.093607: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-23 17:01:56.095003: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-23 17:01:56.095224: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

In [2]:
model= m.NN_Model()
model.get_data("laser_log/laser_log2203.csv")

beta=[0.4]
drop=[0.6]
epochs=200
batch_size=16

group_epochs=1

results=[]
min_rot_error=0.5

for i in beta:
    for j in drop:
        #with sess.as_default():
            model.model_define(drop_rate=j)
            model.set_beta(i)
            model.model_compile("adam",model.pos_loss,['mean_absolute_error'])
            for step in range(epochs//group_epochs):
                history = model.model_run(group_epochs,batch_size, verbose=1)
                pos_mse,rot_mse,pos_mae,rot_mae = model.get_error(verbose=1)
                results.append({"beta":i,"drop-out":j,"epochs":(step+1)*group_epochs,"history":history,"position squared error":pos_mse, "rotation squared error":rot_mse, "position absolute error":pos_mae, "rotation absolute error":rot_mae})  
                if pos_mae < min_rot_error:
                    model.model_cnn.save_weights("models/cp-epoch-train.ckpt")
                    min_rot_error=pos_mae

2022-03-23 17:03:07.680437: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100


130/130 [==============================] - 11s 68ms/step - loss: 7.5515 - mean_absolute_error: 5.7338 - val_loss: 2.1333 - val_mean_absolute_error: 1.5997
position mean square error: 5.837797316356719 -- rotation mean square error: 2.7713409086743828 -- position mean absolute error: 1.7514039820758736 -- rotation mean absolute error: 1.2963565107860753
130/130 [==============================] - 8s 65ms/step - loss: 1.9726 - mean_absolute_error: 1.4736 - val_loss: 1.9973 - val_mean_absolute_error: 1.4679
position mean square error: 5.901837660940072 -- rotation mean square error: 1.5693736092949075 -- position mean absolute error: 1.752193059561459 -- rotation mean absolute error: 0.8992996775345535
130/130 [==============================] - 9s 66ms/step - loss: 1.5454 - mean_absolute_error: 1.1469 - val_loss: 1.7296 - val_mean_absolute_error: 1.2948
position mean square error: 4.0182570333456775 -- rotation mean square error: 1.4833620142652948 -- position mean absolute error: 1.449615

2022-03-23 17:05:54.412851: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 188743680 exceeds 10% of free system memory.
2022-03-23 17:05:54.535388: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 188743680 exceeds 10% of free system memory.
2022-03-23 17:05:54.654417: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 188743680 exceeds 10% of free system memory.


130/130 [==============================] - 9s 67ms/step - loss: 0.4867 - mean_absolute_error: 0.3601 - val_loss: 0.6609 - val_mean_absolute_error: 0.4887
position mean square error: 0.6427393340161044 -- rotation mean square error: 0.2665857587320845 -- position mean absolute error: 0.5790970038873744 -- rotation mean absolute error: 0.3080337477042543
130/130 [==============================] - 8s 64ms/step - loss: 0.4737 - mean_absolute_error: 0.3507 - val_loss: 0.5655 - val_mean_absolute_error: 0.4200
position mean square error: 0.43007265763717917 -- rotation mean square error: 0.25430509580222416 -- position mean absolute error: 0.46431836563011064 -- rotation mean absolute error: 0.33147934349991787


2022-03-23 17:06:17.410169: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 188743680 exceeds 10% of free system memory.
2022-03-23 17:06:17.529115: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 188743680 exceeds 10% of free system memory.


130/130 [==============================] - 9s 66ms/step - loss: 0.4582 - mean_absolute_error: 0.3397 - val_loss: 0.6611 - val_mean_absolute_error: 0.4847
position mean square error: 0.7566190875110017 -- rotation mean square error: 0.26501596916172576 -- position mean absolute error: 0.5664366125375269 -- rotation mean absolute error: 0.32120157161489005
130/130 [==============================] - 8s 64ms/step - loss: 0.4427 - mean_absolute_error: 0.3280 - val_loss: 0.6557 - val_mean_absolute_error: 0.4815
position mean square error: 0.6206051315505738 -- rotation mean square error: 0.279703791976315 -- position mean absolute error: 0.5467998006023006 -- rotation mean absolute error: 0.3509412915524572
130/130 [==============================] - 9s 66ms/step - loss: 0.4402 - mean_absolute_error: 0.3270 - val_loss: 0.6572 - val_mean_absolute_error: 0.4914
position mean square error: 0.5892576780824019 -- rotation mean square error: 0.22863393570269572 -- position mean absolute error: 0.59

In [4]:
with open('test46k.csv', 'w') as f:
    f.write("beta;dropout;epochs;pos_mse;pos_mae;rot_mse;rot_mae\n")
    for item in results:
        f.write("{};{};{};{};{};{};{}\n".format(item["beta"],item["drop-out"],item["epochs"],item["position squared error"],item["position absolute error"],item["rotation squared error"],item["rotation absolute error"]))

#Valutation

In [3]:
model.model_cnn.load_weights("models/cp-epoch-train.ckpt")

2022-03-23 14:17:23.462731: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open models/cp-epoch-train.ckpt: FAILED_PRECONDITION: models/cp-epoch-train.ckpt; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


In [4]:
data_test = m.DAO("laser_log/laser_log2203w.csv")

data_test.read()
data_test.divide_data(0.25)

data_tensor_test=m.DAO()

data_tensor_test.TR_features=tf.reshape(data_test.TR_features,[-1,720,1,1])
data_tensor_test.TS_features=tf.reshape(data_test.TS_features,[-1,720,1,1])
data_tensor_test.TR_targets=tf.reshape(data_test.TR_targets,[-1,3])
data_tensor_test.TS_targets=tf.reshape(data_test.TS_targets,[-1,3])



In [26]:
result = model.model_cnn.predict(data_tensor_test.TS_features)
pos_test = result[:,0:2]
rot_test = result[:,2]
pos_real = np.array([data_test.TS_targets["pos_x"].to_list(),data_test.TS_targets["pos_y"].to_list()]).T
rot_real = (data_test.TS_targets["pos_yaw"]).to_numpy()

In [24]:
pos_mae = mean_squared_error(pos_real,pos_test)
rot_mae = mean_absolute_error(rot_real,rot_test)

In [27]:
with open('predict_data2.csv', 'w') as f:
    for i in range(len(rot_real)):
        f.write("{};{};{};{};{};{}\n".format(pos_real[i][0],pos_real[i][1],rot_real[i],pos_test[i][0],pos_test[i][1],rot_test[i]))